# Number of Non-Directed Interactions

In [3]:
from pip._internal import main as pip
try:
    import csv
except ImportError:
    pip(['install', 'csv'])
    import csv
try:
    from SPARQLWrapper import SPARQLWrapper, JSON
except  ImportError:
    pip(['install', 'sparqlwrapper'])
    from SPARQLWrapper import SPARQLWrapper, JSON
    
import pandas
    
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
NonDirected = pandas.DataFrame(columns=['NonDirected Interactions Count'])
pathwayQuery = '''
      SELECT DISTINCT (count(?entity) as ?NonDirectInt)
      WHERE {
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
        MINUS { ?entity a wp:DirectedInteraction .}
      }  
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	NonDirected = NonDirected.append({
		'NonDirected Interactions Count': result["NonDirectInt"]["value"],
	 }, ignore_index=True)

In [4]:
NonDirected

,NonDirected Interactions Count
0,3601


# Total WP Interactions

In [3]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
totalWPInts = pandas.DataFrame(columns=['Total WP Interactions'])
pathwayQuery = '''
      SELECT DISTINCT (count(?entity) as ?InteractionCount)
      WHERE {
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
      }  
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	totalWPInts = totalWPInts.append({
		'Total WP Interactions': result["InteractionCount"]["value"],
	 }, ignore_index=True)

formTotalWPInts = totalWPInts.loc[:, "Total WP Interactions"].apply(pandas.to_numeric, errors = 'ignore').sum()

In [4]:
totalWPInts

,Total WP Interactions
0,15077


# Count of Interaction Types

In [18]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntTypesCount = pandas.DataFrame(columns=['Interaction Type', 'Interaction Count'])

pathwayQuery = '''
      SELECT (substr(str(?o),41) as ?IntType) (COUNT(?o) as ?IntCount)
      WHERE {
        ?entity a ?o . 
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
      } ORDER BY DESC(?IntCount)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntTypesCount = IntTypesCount.append({
		'Interaction Type': result["IntType"]["value"],
        'Interaction Count': result["IntCount"]["value"],
	 }, ignore_index=True)
    
IntCountInteger = IntTypesCount.loc[:, "Interaction Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 

In [19]:
IntTypesCount

,Interaction Type,Interaction Count
0,Interaction,15077
1,DirectedInteraction,11476
2,Conversion,1362
3,Catalysis,1166
4,Inhibition,1072
5,Binding,850
6,ComplexBinding,850
7,Stimulation,718
8,TranscriptionTranslation,254


# Interaction Count with Unspecified Type

In [7]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
Nonspecific = pandas.DataFrame(columns=['NonSpecified Interaction Type Count'])

pathwayQuery = '''
      SELECT DISTINCT (count(?entity) as ?NonSpecific)
      WHERE {
        ?entity a wp:Interaction . 
        ?entity dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
        MINUS { 
        { ?entity a wp:DirectedInteraction .  }
          UNION
        { ?entity a wp:ComplexBinding .  }
          UNION
        { ?entity a wp:Binding .  }
        }
      }  
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	Nonspecific = Nonspecific.append({
		'NonSpecified Interaction Type Count': result["NonSpecific"]["value"],
	 }, ignore_index=True)

In [8]:
Nonspecific

,NonSpecified Interaction Type Count
0,2751


# Participants for Interactions: 

In [5]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
parts4Ints = pandas.DataFrame(columns=['Interaction', 'Interaction Type', 'Interaction Participants'])


pathwayQuery = '''

    SELECT DISTINCT (substr(str(?interaction),37 ) as ?interactionId ) (substr(str(?intType), 41 ) as ?types) (GROUP_CONCAT(DISTINCT substr(str(?partType),41), ", ") AS ?participants)  WHERE {
       ?interaction a wp:Interaction ;
           dcterms:isPartOf ?pathway ;
           a ?intType .
       ?interaction wp:participants ?participant .
       ?participant a ?partType .
       ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .

       FILTER (?partType != wp:DataNode)
       FILTER (?intType != wp:Interaction)
       FIlTER (?intType != wp:DirectedInteraction)
    } 
    LIMIT 20
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    parts4Ints = parts4Ints.append({
        'Interaction': result["interactionId"]["value"],
        'Interaction Type': result["types"]["value"],
        'Interaction Participants': result["participants"]["value"],
       
    }, ignore_index=True)

In [6]:
parts4Ints

,Interaction,Interaction Type,Interaction Participants
0,WP3925_r96439/WP/Interaction/iddd9403d6_2,Catalysis,"DirectedInteraction, GeneProduct, Interaction"
1,WP15_r98241/WP/Interaction/ec5e2,Stimulation,Metabolite
2,WP1422_r102322/WP/Interaction/id69c02e37,Inhibition,"Conversion, DirectedInteraction, Interaction, ..."
3,WP3959_r101991/WP/Interaction/idf1e7832f,Stimulation,"GeneProduct, Protein"
4,WP4018_r98359/WP/Interaction/idb133fbea,TranscriptionTranslation,"GeneProduct, Protein"
5,WP4008_r96448/ComplexBinding/af036,Binding,"Complex, GeneProduct, Metabolite, Protein"
6,WP697_r96367/WP/Interaction/d9db0,Conversion,Metabolite
7,WP1981_r96314/WP/Interaction/e58fe,Catalysis,"DirectedInteraction, GeneProduct, Interaction"
8,WP1422_r102322/WP/Interaction/id12d8a1bc,Conversion,Metabolite
9,WP3925_r96439/WP/Interaction/id17d9ae_1,Catalysis,"DirectedInteraction, GeneProduct, Interaction"


# Interaction counts by participants

In [11]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntCounts4Parts = pandas.DataFrame(columns=['Interaction Participants', 'Interaction Count'])

pathwayQuery = '''

    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT ?participants (COUNT(?interaction) AS ?count) WHERE {
      SELECT ?interaction (GROUP_CONCAT(substr(str(?partType),41), ", ") AS ?participants) WHERE {
        ?interaction a wp:Interaction . 
        ?interaction wp:participants ?participant . 
        ?participant a ?partType .
        ?interaction dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
        FILTER (?partType != wp:DataNode)
      } GROUP BY ?interaction
    } GROUP BY ?participants
      ORDER BY DESC(?count)
      
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    IntCounts4Parts = IntCounts4Parts.append({
        'Interaction Participants': result["participants"]["value"],
        'Interaction Count': result["count"]["value"],
       
    }, ignore_index=True)

In [12]:
IntCounts4Parts

,Interaction Participants,Interaction Count
0,"Metabolite, Metabolite",2664
1,"GeneProduct, GeneProduct",1934
2,"Protein, GeneProduct, Protein, GeneProduct",1369
3,"GeneProduct, Protein, GeneProduct",602
4,"Protein, GeneProduct",539
5,Metabolite,519
6,GeneProduct,502
7,"Protein, GeneProduct, GeneProduct",441
8,"GeneProduct, DirectedInteraction, Interaction",371
9,Protein,339


# Identifier IDs by data source

In [13]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntIDs = pandas.DataFrame(columns=['Database Source', 'Interactions Counted'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT Distinct (count(?Interaction) as ?IntCount) ?database 
       WHERE {
        ?Interaction a gpml:Interaction ;
            gpml:xrefDataSource ?database .
        ?Interaction dcterms:isPartOf ?pathway .
        ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          
       } ORDER BY DESC(?IntCount)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntIDs = IntIDs.append({
		'Database Source': result["database"]["value"],
        'Interactions Counted': result["IntCount"]["value"],
	 }, ignore_index=True)

In [14]:
IntIDs

,Database Source,Interactions Counted
0,Uniprot-TrEMBL,213
1,Rhea,112
2,KEGG Pathway,28
3,pato,8
4,KEGG Compound,7
5,ChEBI,6
6,Reactome,3
7,KEGG Reaction,3
8,XMetDB,2
9,SPIKE,2


# GPML RDF Interactions WITHOUT a WP RDF equivalent

In [15]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
noWPINT = pandas.DataFrame(columns=['Interactions Counted'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT (COUNT(?Interaction) AS ?count) 
    WHERE {

          ?Interaction a gpml:Interaction .
          ?Interaction dcterms:isPartOf ?pathway .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          MINUS {?wpInteraction wp:isAbout ?Interaction}

        } 
          ORDER BY DESC(?count)
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	noWPINT = noWPINT.append({
        'Interactions Counted': result["count"]["value"],
	 }, ignore_index=True)

In [16]:
noWPINT

,Interactions Counted
0,12162


# Coverage % for GPML Interaction with Equivalent WP RDF out of Total GPML Interactions

In [24]:
IntTable = IntCountInteger
wpInteractionType = IntTable
noWPIntTable = noWPINT.loc[:,'Interactions Counted']
noWPIntIneger = noWPIntTable.apply(pandas.to_numeric, errors = 'ignore').sum()
totalWPInt = noWPIntIneger + wpInteractionType
coverageInt = (wpInteractionType / totalWPInt) * 100
formPercentInt = str(round(coverageInt, 3)) + "% of GPML Interactions with an WP equivalent"

In [25]:
formPercentInt

'72.966% of GPML Interactions with an WP equivalent'

In [26]:
percentWoInt = 100 - coverageInt
str(round(percentWoInt, 3)) + "% of GPML Interactions without a WP equivalent"

'27.034% of GPML Interactions without a WP equivalent'

# Total GPML Interactions

In [27]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
GPMLCountTotal = pandas.DataFrame(columns=['Interactions Count'])

pathwayQuery = '''
    PREFIX gpml:    <http://vocabularies.wikipathways.org/gpml#>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dc:      <http://purl.org/dc/elements/1.1/>
    PREFIX rdf:     <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

    SELECT DISTINCT (COUNT(?Interaction) AS ?count) 
    WHERE {

          ?Interaction a gpml:Interaction .
          ?Interaction dcterms:isPartOf ?pathway .
          ?wpPathway    a wp:Pathway ;
            wp:isAbout ?pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
          
        } 
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	GPMLCountTotal = GPMLCountTotal.append({
        'Interactions Count': result["count"]["value"],
	 }, ignore_index=True)

GPMLCountTotalInt = GPMLCountTotal.loc[:,"Interactions Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 

In [28]:
GPMLCountTotalInt

26954

# % of WP RDF Interactions that are of unspecified type

In [29]:
sparql = SPARQLWrapper("http://sparql.wikipathways.org")
IntTotalCount = pandas.DataFrame(columns=['Interaction Count'])

pathwayQuery = '''
      SELECT DISTINCT (COUNT(?genInteraction) as ?IntCount)
      WHERE {
        ?genInteraction a wp:Interaction .
        ?genInteraction dcterms:isPartOf ?pathway .
        ?pathway    a wp:Pathway ;
            wp:ontologyTag <http://vocabularies.wikipathways.org/wp#Curation:AnalysisCollection> ;
            wp:organismName        "Homo sapiens"^^xsd:string .
      }
    '''
sparql.setQuery(pathwayQuery)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
	IntTotalCount = IntTotalCount.append({
        'Interaction Count': result["IntCount"]["value"],
	 }, ignore_index=True)

NonSpecificTable = Nonspecific.loc[:,"NonSpecified Interaction Type Count"]
NonSpecificInteger = NonSpecificTable.apply(pandas.to_numeric, errors = 'ignore').sum()
TotalInteractions = IntTotalCount.loc[:, "Interaction Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 
percentNonSpecific = (NonSpecificInteger / TotalInteractions) * 100
formPercentNonSpecific = str(round(percentNonSpecific,3)) + '% of WP RDF Interaction with Unspecified Interaction Type'

In [30]:
formPercentNonSpecific

'18.246% of WP RDF Interaction with Unspecified Interaction Type'

In [31]:
percentWSpecificType = 100 - percentNonSpecific
str(round(percentWSpecificType, 3)) + "% of WP RDF Interactions with a specific WP RDF Interaction type"

'81.754% of WP RDF Interactions with a specific WP RDF Interaction type'

# % of NonDirected Interactions

In [32]:
NonDirectedInteger = NonDirected.loc[:, "NonDirected Interactions Count"].apply(pandas.to_numeric, errors = 'ignore').sum() 
percentNonDirected = (NonDirectedInteger / TotalInteractions) * 100
#check this number
formPercentNonDirected = str(round(percentNonDirected, 3)) + '% of Non-directed interaction in WP RDF'

In [33]:
formPercentNonDirected

'23.884% of Non-directed interaction in WP RDF'

In [34]:
ConversionCount = int(IntTypesCount.iloc[0]["Interaction Count"])
BindingCount = int(IntTypesCount.iloc[1]["Interaction Count"])
InteractionWPRDFCount = int(IntTypesCount.iloc[2]["Interaction Count"])
DirectedWPRDFCount = int(IntTypesCount.iloc[3]["Interaction Count"])
CatalysisCount = int(IntTypesCount.iloc[4]["Interaction Count"])
TranscrTranslaCount = int(IntTypesCount.iloc[5]["Interaction Count"])
complexCount = int(IntTypesCount.iloc[6]["Interaction Count"])
InhibitionCount = int(IntTypesCount.iloc[7]["Interaction Count"])
stimulationCount = int(IntTypesCount.iloc[8]["Interaction Count"])

ConversionCount, BindingCount, InteractionWPRDFCount, DirectedWPRDFCount, CatalysisCount, TranscrTranslaCount, complexCount, InhibitionCount, stimulationCount

(15077, 11476, 1362, 1166, 1072, 850, 850, 718, 254)

In [35]:
IntTypesCount

,Interaction Type,Interaction Count
0,Interaction,15077
1,DirectedInteraction,11476
2,Conversion,1362
3,Catalysis,1166
4,Inhibition,1072
5,Binding,850
6,ComplexBinding,850
7,Stimulation,718
8,TranscriptionTranslation,254


In [36]:
#noWPIntIneger without eq
#GPMLCountTotalInt GPML Total
percentComplex = (complexCount / GPMLCountTotalInt ) * 100
formpercentComplex = str(round(percentComplex, 3)) + '% of complex interactions out of all GPML Interactions'
percentComplexWPRDF = (complexCount / formTotalWPInts) * 100
formpercentComplexWPRDF = str(round(percentComplexWPRDF, 3)) + '% of complex interactions out of all WP Interactions'
formpercentComplex,  formpercentComplexWPRDF

('3.154% of complex interactions out of all GPML Interactions',
 '5.638% of complex interactions out of all WP Interactions')

In [37]:
#complexCount = int(IntTypesCount.iloc[6]["Interaction Count"])
PercentConversionCount = (ConversionCount / formTotalWPInts) * 100
formPercentConversionCount = str(round(PercentConversionCount, 3)) + '% of conversion interactions out of all WP Interactions'
PercentBindingCount = (BindingCount / formTotalWPInts) * 100
formPercentBindingCount = str(round(PercentBindingCount, 3)) + '% of binding interactions out of all WP Interactions'
formPercentConversionCount, formPercentBindingCount

('100.0% of conversion interactions out of all WP Interactions',
 '76.116% of binding interactions out of all WP Interactions')

In [38]:
IntTypesCount

,Interaction Type,Interaction Count
0,Interaction,15077
1,DirectedInteraction,11476
2,Conversion,1362
3,Catalysis,1166
4,Inhibition,1072
5,Binding,850
6,ComplexBinding,850
7,Stimulation,718
8,TranscriptionTranslation,254


In [39]:
totalWPInts, GPMLCountTotalInt

(  Total WP Interactions
 0                 15077, 26954)